## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-02-10-52-59 +0000,nypost,Shark attacks 8-year-old boy spearfishing in F...,https://nypost.com/2025/09/02/us-news/shark-at...
1,2025-09-02-10-48-22 +0000,nyt,Trump Says He’ll Give Giuliani the Presidentia...,https://www.nytimes.com/2025/09/02/us/politics...
2,2025-09-02-10-45-30 +0000,bbc,"Hundreds killed in Sudan landslide, UN says",https://www.bbc.com/news/articles/cdj2jygzzk9o...
3,2025-09-02-10-44-01 +0000,nyt,Kraft Heinz to Break Up Its Food Businesses,https://www.nytimes.com/2025/09/02/business/kr...
4,2025-09-02-10-37-27 +0000,bbc,Zack Polanski elected leader of the Green Party,https://www.bbc.com/news/articles/cd0d0d08jnjo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2374/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
14,trump,42
341,china,17
83,earthquake,11
256,dead,10
102,afghanistan,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
171,2025-09-01-20-19-52 +0000,nypost,India’s leader Modi cozies up to Putin and Xi ...,https://nypost.com/2025/09/01/world-news/india...,102
165,2025-09-01-20-43-57 +0000,nypost,Trump awarding Presidential Medal of Freedom t...,https://nypost.com/2025/09/01/us-news/trump-aw...,97
128,2025-09-02-00-05-47 +0000,nypost,Bessent says Trump admin willing to hit Russia...,https://nypost.com/2025/09/01/world-news/besse...,85
132,2025-09-02-00-00-00 +0000,wsj,President Trump faces a challenging September ...,https://www.wsj.com/politics/policy/trumps-sep...,81
266,2025-09-01-11-00-00 +0000,wsj,"The cash benefits of the new tax law, signed b...",https://www.wsj.com/politics/policy/companies-...,76


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
171,102,2025-09-01-20-19-52 +0000,nypost,India’s leader Modi cozies up to Putin and Xi ...,https://nypost.com/2025/09/01/world-news/india...
165,55,2025-09-01-20-43-57 +0000,nypost,Trump awarding Presidential Medal of Freedom t...,https://nypost.com/2025/09/01/us-news/trump-aw...
128,43,2025-09-02-00-05-47 +0000,nypost,Bessent says Trump admin willing to hit Russia...,https://nypost.com/2025/09/01/world-news/besse...
213,42,2025-09-01-16-48-15 +0000,nypost,US companies doing business in China say they’...,https://nypost.com/2025/09/01/business/us-comp...
170,41,2025-09-01-20-20-50 +0000,nypost,"TikTok influencer and her family found dead, w...",https://nypost.com/2025/09/01/world-news/influ...
246,38,2025-09-01-13-53-10 +0000,bbc,BBC reports from scene as Afghanistan earthqua...,https://www.bbc.com/news/videos/c3v3gg6zr3yo?a...
86,37,2025-09-02-05-11-55 +0000,nypost,"6-year-old boy in critical condition, 2 adults...",https://nypost.com/2025/09/02/us-news/2-adults...
57,35,2025-09-02-08-23-36 +0000,nypost,"Sudan landslide kills over 1,000 people and wi...",https://nypost.com/2025/09/02/world-news/sudan...
241,34,2025-09-01-14-06-02 +0000,nypost,European Commission says president Ursula von ...,https://nypost.com/2025/09/01/world-news/europ...
233,34,2025-09-01-14-37-46 +0000,cbc,Israeli tanks advance in Gaza City as scholars...,https://www.cbc.ca/news/world/gaza-city-invasi...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
